<a href="https://colab.research.google.com/github/Amari-Lawal/Data-Science-Projects-/blob/main/NLP%20Practice%20Folder/NLP_Practice_Main.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

NLP Classification Structure:

1.   Tokenizing, getting rid of stop words, lemmas
2.   Vectorise when dealing with normal ML models neural Networks
3.   Count frequency of words
4.   When using LSTM's and Embedding layers instead of Vectorise set into sequences
5.   Set y as dummy variables
6.   Predict and Evaluate
7.   Show Values visually



In [ ]:
# Upload a kaggle.json api key by going into settings of a kaggle account
from google.colab import files
files.upload()

Saving kaggle.json to kaggle (1).json


{'kaggle.json': b'{"username":"stinner999palace","key":"473e7cf39093d8fb6cf2f7dd909f5633"}'}

In [ ]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/ # Makes the directory to store the kaggle api key to be able to access the dataset
!chmod 600 ~/.kaggle/kaggle.json

In [ ]:
!kaggle datasets download -d team-ai/spam-text-message-classification # Downloads dataset from kaggle

spam-text-message-classification.zip: Skipping, found more recently modified local copy (use --force to force download)


In [ ]:
!ls

'Amari Lawal CV.pdf'   kaggle.json  'SPAM text message 20170820 - Data.csv'
'kaggle (1).json'      sample_data   spam-text-message-classification.zip


In [ ]:
import zipfile
zip_ref = zipfile.ZipFile("spam-text-message-classification.zip", "r") # Unzips the zip file
zip_ref.extractall()
zip_ref.close()

In [ ]:
DATA_URL = "/content/SPAM text message 20170820 - Data.csv"
import pandas as pd
df = pd.read_csv(DATA_URL) # Loads in the 
display(df)

,Category,Message
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."
...,...,...
5567,spam,This is the 2nd time we have tried 2 contact u...
5568,ham,Will ü b going to esplanade fr home?
5569,ham,"Pity, * was in mood for that. So...any other s..."
5570,ham,The guy did some bitching but I acted like i'd...


In [ ]:
import tensorflow as tf

In [ ]:
import spacy
import pandas as pd
import numpy as np
#!python -m spacy download en_core_web_lg
nlp = spacy.load('en_core_web_lg') 
#nlp = spacy.load('en_core_web_sm')

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer,TfidfVectorizer
from sklearn.base import TransformerMixin
from sklearn.pipeline import Pipeline

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5572 entries, 0 to 5571
Data columns (total 2 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   Category  5572 non-null   object
 1   Message   5572 non-null   object
dtypes: object(2)
memory usage: 87.2+ KB


In [ ]:
# Get all sentences from the dataframe 
# Get tokens of words in each sentence
# Add to the vocab
# Then count frequency of words between all sentences
#parser = spacy.lang.en.English()
#parser = spacy.lang.en.English()
def tokenizer_extraction(dataset):
  # Imports Counter Function
  from collections import Counter
  my_tokens = nlp(dataset)
  # Creates set which you add the vocab to and makes sure that there are no duplicates
  vocab = set() # However this means the vocab won't be in order and the indexes may change, INDEXES is the data format we will use to input into model/ 
  # Gets the tokens from the document getting rid of stop words, punctuation, digits and inculding lemmas
  my_tokens = [token.lemma_.lower().strip() for token in my_tokens if not token.is_stop and not token.is_punct and not token.is_digit and not token.is_currency and token.is_ascii] # May want to get rid of non unicode values with ord(token) < 128 on ascii table.
  # Adds the tokens in words to vocab set.
  for i in my_tokens:
    vocab.add(i)
  # Counts the word frequency 
  word_freq = Counter(my_tokens)
  common_words = word_freq.most_common(len(dataset))


  print(f'Vocab Size: {len(vocab)} ')
  print(f'Vocab Words: {vocab}')
  print("Word Frequency:",common_words)
  return my_tokens

In [ ]:
#vocab_lis, word_lis, words = tokenizer_extraction(df,1)

In [ ]:
# Sets pridictors class with TransformMixin characteristics, with fit and fit_transform/transform allows you to transform data when called.
class predictors(TransformerMixin): # 
  def transform(self,X,**transform_params):
    # Cleaning Text
    return [clean_text(text) for text in X]
  def fit(self,X,y=None,**fit_params):
    return self
    def get_params(self,deep=True):
      return {}
def clean_text(text):
    return text.strip().lower()

In [ ]:
bow_vector = CountVectorizer(tokenizer = tokenizer_extraction,ngram_range=(1,1)) # Function that tokenises with token_extractor and vectorises at once

In [ ]:
tfidf_vector = TfidfVectorizer(tokenizer=tokenizer_extraction) # Other vectorixer Method

In [ ]:
# Gets Data and splits
from sklearn.model_selection import train_test_split
X = df["Message"]
df_dum = pd.get_dummies(df["Category"])
ylabels = df_dum['spam'] # Set values as dummy variables
X_train, X_test, y_train, y_test = train_test_split(X,ylabels,test_size=0.3)

In [ ]:
# Creates Pipeline for 
from sklearn.linear_model import LogisticRegression
classifier = LogisticRegression()
# wrap the model using the function you created
#clf = tf.keras.wrappers.scikit_learn.KerasRegressor(build_fn=create_model,verbose=0)

# When using Pipeline step has to have fit or fit_transform in the objects inserted. By using Transformer MixIn
pipe = Pipeline([("cleaner",predictors()),
                 ("vectorizer",bow_vector),
                 ("classifier",classifier)
                 ])
pipe.fit(X_train,y_train)


Streaming output truncated to the last 5000 lines.
Vocab Words: {'finish', 'work'}
Word Frequency: [('finish', 1), ('work', 1)]
Vocab Size: 5 
Vocab Words: {'feel', 'lennon', 'ok', 'like', 'john'}
Word Frequency: [('ok', 1), ('feel', 1), ('like', 1), ('john', 1), ('lennon', 1)]
Vocab Size: 6 
Vocab Words: {'rub', 'wound', 'salt', 'get', 'like', 'open'}
Word Frequency: [('get', 1), ('salt', 1), ('rub', 1), ('open', 1), ('wound', 1), ('like', 1)]
Vocab Size: 8 
Vocab Words: {'admin', 'reach', 'b', 'u', 'build', 'e', 'earlier', 'slightly'}
Word Frequency: [('e', 1), ('admin', 1), ('build', 1), ('b', 1), ('slightly', 1), ('earlier', 1), ('u', 1), ('reach', 1)]
Vocab Size: 6 
Vocab Words: {'number', 'leave', 'miss', 'alert', 'call', 'message'}
Word Frequency: [('miss', 1), ('alert', 1), ('number', 1), ('call', 1), ('leave', 1), ('message', 1)]
Vocab Size: 9 
Vocab Words: {'', 'pls', 'min', 'lt;#&gt', 'feed', 'later', 'prometazine', '5mls', 'syrup'}
Word Frequency: [('pls', 1), ('prometazine

Pipeline(memory=None,
         steps=[('cleaner', <__main__.predictors object at 0x7fe8dad763d0>),
                ('vectorizer',
                 CountVectorizer(analyzer='word', binary=False,
                                 decode_error='strict',
                                 dtype=<class 'numpy.int64'>, encoding='utf-8',
                                 input='content', lowercase=True, max_df=1.0,
                                 max_features=None, min_df=1,
                                 ngram_range=(1, 1), preprocessor=None,
                                 stop_words=None, strip_accents=None,
                                 t...\\w+\\b',
                                 tokenizer=<function tokenizer_extraction at 0x7fe8db942950>,
                                 vocabulary=None)),
                ('classifier',
                 LogisticRegression(C=1.0, class_weight=None, dual=False,
                                    fit_intercept=True, intercept_scaling=1,
             

In [ ]:
from sklearn import metrics
predicted = pipe.predict(X_test)
print(f'-------------------------------------------------------------------------')
print(f'Logistsic Regression Accuracy: {metrics.accuracy_score(y_test,predicted)}')
print(f'Logistic Regression Precision: {metrics.precision_score(y_test,predicted)}')
print(f'Logistic Regression Recall: {metrics.recall_score(y_test,predicted)}')

Streaming output truncated to the last 5000 lines.
Word Frequency: [('', 2), ('want', 1), ('lt;#&gt', 1), ('r', 1), ('da:)do', 1)]
Vocab Size: 3 
Vocab Words: {'wait', 'letter', 'sir'}
Word Frequency: [('sir', 1), ('wait', 1), ('letter', 1)]
Vocab Size: 2 
Vocab Words: {'hello', 'madam'}
Word Frequency: [('hello', 1), ('madam', 1)]
Vocab Size: 3 
Vocab Words: {'willing', 'r', 'pay'}
Word Frequency: [('r', 1), ('willing', 1), ('pay', 1)]
Vocab Size: 4 
Vocab Words: {'', 'go', 'x2', 'lt;#&gt'}
Word Frequency: [('x2', 1), ('', 1), ('lt;#&gt', 1), ('go', 1)]
Vocab Size: 2 
Vocab Words: {'marry', 'tell'}
Word Frequency: [('tell', 1), ('marry', 1)]
Vocab Size: 4 
Vocab Words: {'fine', 'shall', 'avalarr', 'hollalater'}
Word Frequency: [('shall', 1), ('fine', 1), ('avalarr', 1), ('hollalater', 1)]
Vocab Size: 7 
Vocab Words: {'clover', 'saying', 'pub', 'be', 'happy', 'pig', 'guess'}
Word Frequency: [('be', 2), ('guess', 1), ('pub', 1), ('happy', 1), ('pig', 1), ('clover', 1), ('saying', 1)]
Vo

In [ ]:
for i in range(0,len(X_test)):
  if predicted[i] == 1:
    print(f'Spam: {X_test.iloc[i]}')
  elif predicted[i] == 0:
    print(f'Not Spam: {X_test.iloc[i]}')


Not Spam: I want to tel u one thing u should not mistake me k THIS IS THE MESSAGE THAT YOU SENT:)
Spam: Todays Vodafone numbers ending with 4882 are selected to a receive a £350 award. If your number matches call 09064019014 to receive your £350 award.
Not Spam: Oh:)as usual vijay film or its different?
Not Spam: Free any day but i finish at 6 on mon n thurs...
Not Spam: I guess that's why you re worried. You must know that there's a way the body repairs itself. And i'm quite sure you shouldn't worry. We'll take it slow. First the tests, they will guide when your ovulation is then just relax. Nothing you've said is a reason to worry but i.ll keep on followin you up.
Not Spam: One day a crab was running on the sea shore..The waves came n cleared the footprints of the crab.. Crab asked: being my frnd y r u clearing my beautiful footprints? Waves replied: A fox was following ur footprints to catch you! thats y i cleared it off:) frndsship never lets u dwn :-) GUD nyt..
Not Spam: I want  &

In [ ]:
#TFID_Vectorisation
# wrap the model using the function you created
#clf = tf.keras.wrappers.scikit_learn.KerasRegressor(build_fn=create_model,verbose=0)

# When using Pipeline step has to have fit or fit_transform in the objects inserted. By using Transformer MixIn
pipe_tf = Pipeline([("cleaner",predictors()),
                 ("vectorizer",tfidf_vector),
                 ("classifier",classifier)
                 ])
pipe_tf.fit(X_train,y_train)

Streaming output truncated to the last 5000 lines.
Vocab Words: {'finish', 'work'}
Word Frequency: [('finish', 1), ('work', 1)]
Vocab Size: 5 
Vocab Words: {'feel', 'lennon', 'ok', 'like', 'john'}
Word Frequency: [('ok', 1), ('feel', 1), ('like', 1), ('john', 1), ('lennon', 1)]
Vocab Size: 6 
Vocab Words: {'rub', 'wound', 'salt', 'get', 'like', 'open'}
Word Frequency: [('get', 1), ('salt', 1), ('rub', 1), ('open', 1), ('wound', 1), ('like', 1)]
Vocab Size: 8 
Vocab Words: {'admin', 'reach', 'b', 'u', 'build', 'e', 'earlier', 'slightly'}
Word Frequency: [('e', 1), ('admin', 1), ('build', 1), ('b', 1), ('slightly', 1), ('earlier', 1), ('u', 1), ('reach', 1)]
Vocab Size: 6 
Vocab Words: {'number', 'leave', 'miss', 'alert', 'call', 'message'}
Word Frequency: [('miss', 1), ('alert', 1), ('number', 1), ('call', 1), ('leave', 1), ('message', 1)]
Vocab Size: 9 
Vocab Words: {'', 'pls', 'min', 'lt;#&gt', 'feed', 'later', 'prometazine', '5mls', 'syrup'}
Word Frequency: [('pls', 1), ('prometazine

Pipeline(memory=None,
         steps=[('cleaner', <__main__.predictors object at 0x7fe8d97e04d0>),
                ('vectorizer',
                 TfidfVectorizer(analyzer='word', binary=False,
                                 decode_error='strict',
                                 dtype=<class 'numpy.float64'>,
                                 encoding='utf-8', input='content',
                                 lowercase=True, max_df=1.0, max_features=None,
                                 min_df=1, ngram_range=(1, 1), norm='l2',
                                 preprocessor=None, smooth_idf=True,
                                 stop_wor...
                                 tokenizer=<function tokenizer_extraction at 0x7fe8db942950>,
                                 use_idf=True, vocabulary=None)),
                ('classifier',
                 LogisticRegression(C=1.0, class_weight=None, dual=False,
                                    fit_intercept=True, intercept_scaling=1,
        

In [ ]:
#TFID_Vecorisation
from sklearn import metrics
predicted_tf = pipe_tf.predict(X_test)
print(f'-------------------------------------------------------------------------')
print(f'Logistsic Regression Accuracy: {metrics.accuracy_score(y_test,predicted_tf)}')
print(f'Logistic Regression Precision: {metrics.precision_score(y_test,predicted_tf)}')
print(f'Logistic Regression Recall: {metrics.recall_score(y_test,predicted_tf)}')

Streaming output truncated to the last 5000 lines.
Word Frequency: [('', 2), ('want', 1), ('lt;#&gt', 1), ('r', 1), ('da:)do', 1)]
Vocab Size: 3 
Vocab Words: {'wait', 'letter', 'sir'}
Word Frequency: [('sir', 1), ('wait', 1), ('letter', 1)]
Vocab Size: 2 
Vocab Words: {'hello', 'madam'}
Word Frequency: [('hello', 1), ('madam', 1)]
Vocab Size: 3 
Vocab Words: {'willing', 'r', 'pay'}
Word Frequency: [('r', 1), ('willing', 1), ('pay', 1)]
Vocab Size: 4 
Vocab Words: {'', 'go', 'x2', 'lt;#&gt'}
Word Frequency: [('x2', 1), ('', 1), ('lt;#&gt', 1), ('go', 1)]
Vocab Size: 2 
Vocab Words: {'marry', 'tell'}
Word Frequency: [('tell', 1), ('marry', 1)]
Vocab Size: 4 
Vocab Words: {'fine', 'shall', 'avalarr', 'hollalater'}
Word Frequency: [('shall', 1), ('fine', 1), ('avalarr', 1), ('hollalater', 1)]
Vocab Size: 7 
Vocab Words: {'clover', 'saying', 'pub', 'be', 'happy', 'pig', 'guess'}
Word Frequency: [('be', 2), ('guess', 1), ('pub', 1), ('happy', 1), ('pig', 1), ('clover', 1), ('saying', 1)]
Vo

In [ ]:
#TFID Vectorisation
for i in range(0,len(X_test)):
  if predicted_tf[i] == 1:
    print(f'Spam: {X_test.iloc[i]}')
  elif predicted_tf[i] == 0:
    print(f'Not Spam: {X_test.iloc[i]}')

Not Spam: I want to tel u one thing u should not mistake me k THIS IS THE MESSAGE THAT YOU SENT:)
Not Spam: Todays Vodafone numbers ending with 4882 are selected to a receive a £350 award. If your number matches call 09064019014 to receive your £350 award.
Not Spam: Oh:)as usual vijay film or its different?
Not Spam: Free any day but i finish at 6 on mon n thurs...
Not Spam: I guess that's why you re worried. You must know that there's a way the body repairs itself. And i'm quite sure you shouldn't worry. We'll take it slow. First the tests, they will guide when your ovulation is then just relax. Nothing you've said is a reason to worry but i.ll keep on followin you up.
Not Spam: One day a crab was running on the sea shore..The waves came n cleared the footprints of the crab.. Crab asked: being my frnd y r u clearing my beautiful footprints? Waves replied: A fox was following ur footprints to catch you! thats y i cleared it off:) frndsship never lets u dwn :-) GUD nyt..
Not Spam: I wan

In [ ]:
def text_predict(input):
  #text = list()
  text = []
  text.append(input)
  pred_text = pipe.predict(text)
  if pred_text[0] == 1:
    print(f'--------------------------------------------------------------------------')
    print(f'Spam: {text[0]}')
  elif pred_text[0] == 0:
    print(f'---------------------------------------------------------------------------')
    print(f'Not Spam: {text[0]}')

text_predict(input())
#list(input())

KeyboardInterrupt: ignored

TODO LSTM Network
1. Next sequence data without vectorizing
2. Feed into Word Embedding then Neural Network

In [ ]:
def Lstm():
  layer = Embedding(input_dim= sentence,output_dim=50,input_length=max_len)  #vocab, vector space ;output of the Embedding layer is a 2D vector
  layer = LSTM(64)(layer)
  layer = Dense(256,name='FC1',activation='relu')(layer)
  layer = Dropout(0.5)(layer)
  layer = Dense(2,name='out_layer',activation='sigmoid')(layer)
  model = Model(inputs=inputs,outputs=layer)
  return model

In [ ]:
#from keras.preprocessing.text import Tokenizer
#from keras.preprocessing import sequence
#from keras.utils import to_categorical
#from keras.callbacks import EarlyStopping

#max_words = 1000
#max_len = 150
#tok = Tokenizer(num_words=max_words)
#tok.fit_on_texts(X_train)

#sequences = tok.texts_to_sequences(X_train)
#sequences_matrix = sequence.pad_sequences(sequences,maxlen=max_len)

#max_words = 1000
#max_len = 150
#tf.keras.preprocessing.sequence.pad_sequences(X_train,maxlen=150)
#tok = Tokenizer(num_words=max_words)
#tok.fit_on_texts(X_train)


# X_train Transformations from cleaning data from predictors
#pred_train = predictors()
#X_train = pred_train.transform(X_train)
#X_train = bow_vector.fit_transform(X_train)

#X_test = pred_train.transform(X_test)
#X_test = bow_vector.fit_transform(X_test)

